# Azure OpenAI GPT Playground

In [74]:
import json
import openai

In [75]:
def prompt_completion(messages, model, temperature=0.3, max_tokens=500, top_p=0.95, stop=None):
    response = openai.ChatCompletion.create(
                  engine=model,
                  messages=messages,
                  temperature=temperature,
                  max_tokens=max_tokens,
                  top_p=top_p,
                  stop=stop
                  )
    return response['choices'][0]['message']['content'].strip()

def preprocess_prompt(prompt, instruction=None):
    system_message = "You are an AI assistant that helps people find information." if instruction is None else instruction
    messages=[{"role": "system", "content": system_message}, {"role": "user", "content": prompt}]
    return messages

## Configure Azure OpenAI API

In [76]:
with open('../configs/config.json', 'r') as f:
    config = json.load(f)

In [77]:
openai.api_key = config['OPENAI_API_KEY']
openai.api_base = config['OPENAI_API_BASE']
openai.api_type = config['OPENAI_API_TYPE']
openai.api_version = config['OPENAI_API_VERSION']
model_name = config['OPENAI_MODEL_NAME']

## Extract Entities

In [118]:
prompt = ("Extract the entities from the following text: "
              "The new iPhone model is set to be released next month. "
              "It has been highly anticipated by Apple fans and is expected "
              "to feature a larger screen and improved camera.")

messages = preprocess_prompt(prompt)
response = prompt_completion(messages, model=model_name)
print(response)

Entities:
- iPhone (product)
- Apple (brand)
- screen (feature)
- camera (feature)


In [117]:
prompt = ("Extract the entities from the following text and produce output in the json format: "
              "The new iPhone model is set to be released next month. "
              "It has been highly anticipated by Apple fans and is expected "
              "to feature a larger screen and improved camera.")

messages = preprocess_prompt(prompt)
response = prompt_completion(messages, model=model_name)
print(response)

{
  "entities": [
    {
      "entity": "iPhone",
      "type": "product"
    },
    {
      "entity": "Apple",
      "type": "company"
    },
    {
      "entity": "screen",
      "type": "feature"
    },
    {
      "entity": "camera",
      "type": "feature"
    },
    {
      "entity": "next month",
      "type": "time"
    }
  ]
}


In [116]:
prompt = ("""Please summarize the information about the product based on the following: 
            {
              "entities": [
                {
                  "entity": "iPhone",
                  "type": "product"
                },
                {
                  "entity": "Apple",
                  "type": "company"
                },
                {
                  "entity": "next month",
                  "type": "date"
                },
                {
                  "entity": "larger screen",
                  "type": "feature"
                },
                {
                  "entity": "improved camera",
                  "type": "feature"
                }
              ]
            }
        """)

messages = preprocess_prompt(prompt)
response = prompt_completion(messages, model=model_name)
print(response)

The information is about a new iPhone product that will be released by Apple next month. The new iPhone will have a larger screen and an improved camera.


In [115]:
prompt = ("Please provide a sentiment for the following text and produce output as json: "
          "Apple is releasing a new iPhone with a larger screen and improved camera next month.")

messages = preprocess_prompt(prompt)
response = prompt_completion(messages, model=model_name)
print(response)

Sentiment: Neutral

Output:
{
  "text": "Apple is releasing a new iPhone with a larger screen and improved camera next month.",
  "sentiment": "Neutral"
}


### Translation

In [114]:
prompt = ("Please translate the following text in Swiss German: "
          "I have to go shopping so that I can prepare a nine o’clock snack for my son.")

messages = preprocess_prompt(prompt)
response = prompt_completion(messages, model=model_name)
print(response)

Ich muess go schoppe, damit ich es Nüüni-Znüni für min Sohn vorbereite cha.


### Explain solution

In [113]:
prompt = ("Determine if the student's solution is correct or not: "
          "Question: I'm building a solar power installation and I need help working out the financials. "
              "Land costs $100 / square foot "
              "I can buy solar panels for $250 / square foot "
              "I negotiated a contract for maintenance that will cost me a flat $100k per year, and an additional $10 square foot "
          "What is the total cost for the first year of operations as a function of the number of square feet. "
          "Student's Solution: Let x be the size of the installation in square feet. "
          "Costs: "
              "1. Land cost: 100x "
              "2. Solar panel cost: 250x "
              "3. Maintenance cost: 100,000 + 100x "
          "Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000 ")

messages = preprocess_prompt(prompt)
response = prompt_completion(messages, model=model_name)
print(response)

The student's solution is correct.


In [112]:
prompt = ("Solve the following question and explain the reasoning: "
          "Question: I'm building a solar power installation and I need help working out the financials. "
              "Land costs $100 / square foot "
              "I can buy solar panels for $250 / square foot "
              "I negotiated a contract for maintenance that will cost me a flat $100k per year, and an additional $10 square foot "
          "What is the total cost for the first year of operations as a function of the number of square feet. "
          "Solution:"
         )

messages = preprocess_prompt(prompt)
response = prompt_completion(messages, model=model_name)
print(response)

To calculate the total cost for the first year of operations as a function of the number of square feet, we need to consider the cost of land, solar panels, and maintenance.

Let's assume that the number of square feet is represented by "x".

The cost of land would be $100 times the number of square feet, which is 100x.

The cost of solar panels would be $250 times the number of square feet, which is 250x.

The cost of maintenance would be a flat $100k per year, plus $10 per square foot. So the total cost of maintenance would be $100k + $10 times the number of square feet, which is 100k + 10x.

Therefore, the total cost for the first year of operations would be the sum of the cost of land, solar panels, and maintenance. 

Total cost = Cost of land + Cost of solar panels + Cost of maintenance
Total cost = 100x + 250x + (100k + 10x)
Total cost = 360x + 100k

So the total cost for the first year of operations as a function of the number of square feet is 360x + 100k.


### System Instruction

In [111]:
prompt = ("Explain Generative AI in German Language.")

messages = preprocess_prompt(prompt)
response = prompt_completion(messages, model=model_name)
print(response)

Generative AI bezieht sich auf eine Art von künstlicher Intelligenz, die in der Lage ist, neue Daten oder Inhalte zu erstellen, die auf vorhandenen Mustern oder Regeln basieren. Im Gegensatz zu anderen Arten von künstlicher Intelligenz, die darauf ausgelegt sind, Muster in vorhandenen Daten zu erkennen und Vorhersagen zu treffen, kann Generative AI tatsächlich neue Daten generieren, die ähnlich wie die vorhandenen Daten sind. Dies wird durch die Verwendung von Deep-Learning-Modellen und anderen fortgeschrittenen Technologien erreicht, die es der künstlichen Intelligenz ermöglichen, komplexe Muster und Zusammenhänge zu erkennen und zu reproduzieren. Generative AI wird in einer Vielzahl von Anwendungen eingesetzt, darunter Bild- und Spracherkennung, Textgenerierung und sogar Musikkomposition.


In [110]:
prompt = ("Explain Generative AI in German Language.")

messages = preprocess_prompt(prompt, 
                             instruction="You are an AI assistant that helps people find information. Don't speak in German.")
response = prompt_completion(messages, model=model_name)
print(response)

Entschuldigung, ich habe Ihre Anweisungen nicht verstanden. Ich bin ein AI-Assistent, der Menschen bei der Suche nach Informationen hilft, aber ich spreche nicht auf Deutsch. Kann ich Ihnen bei etwas anderem helfen?


In [109]:
prompt = ("Explain Generative AI in German Language.")

messages = preprocess_prompt(prompt, 
                             instruction="You are an AI assistant that helps people find information. Please only converse in English. Say no to other languages.")
response = prompt_completion(messages, model=model_name)
print(response)

I'm sorry, but I can only converse in English as per my programming. Can I assist you with anything else in English?


### Hallucination

Possible solutions:
- Limit the outcome e.g. MCQ, Yes/No
- Specify a specific system role and instruct it not to lie
- Experiment with Temperature
- Transparency and Explainablity
- Human-in-loop

In [108]:
prompt = ("Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie.")

messages = preprocess_prompt(prompt)
response = prompt_completion(messages, model=model_name)
print(response)

The AeroGlide UltraSlim Smart Toothbrush by Boie is a high-tech toothbrush that uses sonic vibrations to clean your teeth. It has a slim design that makes it easy to maneuver and reach all areas of your mouth. The toothbrush also features a smart timer that ensures you brush for the recommended two minutes. The brush head is made of a soft, antimicrobial material that is gentle on your teeth and gums. Additionally, the toothbrush comes with a charging base that can be used to charge the brush between uses. Overall, the AeroGlide UltraSlim Smart Toothbrush by Boie is a great option for anyone looking for a high-quality toothbrush that is both effective and easy to use.


In [106]:
prompt = ("Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie.")

messages = preprocess_prompt(prompt, instruction="You are an AI assistant that helps people find information. Please answer only if you are very sure otherwise say no.")
response = prompt_completion(messages, model=model_name, temperature=0)
print(response)

The AeroGlide UltraSlim Smart Toothbrush by Boie is a toothbrush that uses sonic vibrations to clean teeth. It has a slim design that allows it to reach tight spaces in the mouth, and it comes with a charging base that can be used to charge the toothbrush. The toothbrush also has a smart timer that helps users brush for the recommended two minutes, and it connects to a mobile app that provides feedback on brushing habits. However, as an AI language model, I do not have personal experience with the product, and my information is based on publicly available sources.
